### Vggfaces

The VGG (Visual Geometry Group) architecture is a family of convolutional neural network (CNN) models that have been widely used for image classification tasks. The original VGG architecture, known as VGG16, was introduced by the Visual Geometry Group at Oxford University. It has a simple and uniform architecture with a focus on depth, using 3x3 convolutional filters throughout the network.

If "VGGFaces" is a term used in the context of facial recognition, it might refer to a customized or fine-tuned version of the VGG architecture for face-related tasks. Researchers and practitioners often adapt existing CNN architectures for specific domains, and a model named "VGGFaces" could be designed or trained specifically for facial recognition.

![vgg.jpg](./vggface.png)


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import color,io ,img_as_ubyte , feature
from skimage.feature import graycomatrix,graycoprops
from skimage.measure import shannon_entropy
from scipy.stats import describe
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from google.colab import drive
import random
#drive.mount('/content/drive')


## Loading & scaling the dataset and augmenting the images 

In [2]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(dataset_path):
    images = []
    labels = []

    # Data augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if os.path.isdir(person_path):
            # Get the list of existing images
            image_files = os.listdir(person_path)
            current_image_count = len(image_files)

            # Augment data if the image count is less than 30
            target_image_count = 40
            while current_image_count < target_image_count:
                # Choose a random image from the existing ones
                random_image = image_files[current_image_count % len(image_files)]
                image_path = os.path.join(person_path, random_image)

                # Load the image and reshape it to (1, height, width, channels)
                img = cv2.imread(image_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (224, 224))
                    img = img / 255.0

                    # Reshape to (1, height, width, channels)
                    img = np.expand_dims(img, axis=0)

                    # Perform data augmentation
                    augmented_img_gen = datagen.flow(img, save_prefix='aug', save_format='png')

                    # Generate and append augmented images
                    for _ in range(target_image_count - current_image_count):
                        augmented_img = augmented_img_gen.next()
                        images.append(augmented_img[0])
                        labels.append(person_name)
                        current_image_count += 1
                else:
                    print(f"Error loading image: {image_path}")

            # Load original images without augmentation
            for filename in image_files:
                img_path = os.path.join(person_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (224, 224))
                    img = img / 255.0
                    images.append(img)
                    labels.append(person_name)
                else:
                    print(f"Error loading image: {img_path}")

    return np.array(images), np.array(labels)

# Provide the path to your dataset
dataset_path = 'datasetpro2'
images, labels = load_data(dataset_path)


## spliting and encoding 

In [3]:

# Split the data into training and testing sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(labels)
# Now, x_train, y_train, x_test, y_test contain the split datasets
x_train, x_test, y_train, y_test = train_test_split(images, y_train, test_size=0.2, random_state=42)
# If you want to check the shapes of the arrays
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
#print( x_train[0])
#print( y_train[0])
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (2645, 224, 224, 3)
y_train shape: (2645,)
x_test shape: (662, 224, 224, 3)
y_test shape: (662,)


In [4]:
# Building the vggface Model Architecture

model2 = Sequential()

model2.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), strides=(2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), strides=(2, 2)))
model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), strides=(2, 2)))
model2.add(Flatten())
model2.add(Dense(4096, activation='relu'))
model2.add(Dense(4096, activation='relu'))
model2.add(Dense(55, activation='softmax'))
# Compiling the Model
model2.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
# Display the model summary
model2.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [5]:
vggfaces = model2.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test))

Epoch 1/10
166/166 [==============================] - 1608s 10s/step - loss: 3.8806 - accuracy: 0.1577 - val_loss: 3.8201 - val_accuracy: 0.1390
Epoch 2/10
166/166 [==============================] - 1633s 10s/step - loss: 3.6673 - accuracy: 0.1645 - val_loss: 3.6218 - val_accuracy: 0.1465
Epoch 3/10
166/166 [==============================] - 11192s 68s/step - loss: 3.5520 - accuracy: 0.1656 - val_loss: 3.5752 - val_accuracy: 0.1450
Epoch 4/10
122/166 [=====================>........] - ETA: 6:11:37 - loss: 3.4819 - accuracy: 0.1726

KeyboardInterrupt: 

In [ ]:
#Model Evaluation
loss, accuracy = model2.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

19/19 [==============================] - 4s 197ms/step - loss: 3.3235 - accuracy: 0.5559
Test Loss: 3.3235, Test Accuracy: 0.5559


In [ ]:
train_accuracy = vggfaces.history['accuracy']
val_accuracy = vggfaces.history['val_accuracy']
epochs = range(1,vggfaces.epoch[-1]+2)
plt.plot(epochs, train_accuracy,'g', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()